In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [2]:
os.listdir('/kaggle/input/')

['titanic']

Importing the training set

In [3]:
train = pd.read_csv('/kaggle/input/titanic/train.csv')
print(train.shape)
train.head()

(891, 12)


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
train['Embarked'].mode()

0    S
dtype: object

In [5]:
train['Pclass'].fillna(train['Pclass'].mode(), inplace = True)
train['Sex'].fillna(train['Sex'].mode(), inplace = True)
train['Fare'].fillna(train['Fare'].mean(), inplace = True)
train['Embarked'].fillna(train['Embarked'].mode(), inplace = True)

In [6]:
np.sum(train['Embarked'].isna())

2

Encoding categorical variables - Sex and Emarked - into numerical values

In [7]:
from sklearn.preprocessing import OrdinalEncoder
enc = OrdinalEncoder()

In [8]:
train['Sex'] = enc.fit_transform(train.loc[:, train.columns == 'Sex'])
print(enc.categories_)
train['Embarked'] = enc.fit_transform(train.loc[:, train.columns == 'Embarked'])
print(enc.categories_)

[array(['female', 'male'], dtype=object)]
[array(['C', 'Q', 'S', nan], dtype=object)]


In [9]:
train.describe(include = 'all')

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
count,891.000000,891.000000,891.000000,891,891.000000,714.000000,891.000000,891.000000,891,891.000000,204,889.000000
unique,NaN,NaN,NaN,891,NaN,NaN,NaN,NaN,681,NaN,147,NaN
top,NaN,NaN,NaN,"Braund, Mr. Owen Harris",NaN,NaN,NaN,NaN,347082,NaN,B96 B98,NaN
freq,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,7,NaN,4,NaN
mean,446.000000,0.383838,2.308642,NaN,0.647587,29.699118,0.523008,0.381594,NaN,32.204208,NaN,1.535433
std,257.353842,0.486592,0.836071,NaN,0.477990,14.526497,1.102743,0.806057,NaN,49.693429,NaN,0.792088
min,1.000000,0.000000,1.000000,NaN,0.000000,0.420000,0.000000,0.000000,NaN,0.000000,NaN,0.000000
25%,223.500000,0.000000,2.000000,NaN,0.000000,20.125000,0.000000,0.000000,NaN,7.910400,NaN,1.000000
50%,446.000000,0.000000,3.000000,NaN,1.000000,28.000000,0.000000,0.000000,NaN,14.454200,NaN,2.000000
75%,668.500000,1.000000,3.000000,NaN,1.000000,38.000000,1.000000,0.000000,NaN,31.000000,NaN,2.000000


In [10]:
print('Pclass frequent:', train['Pclass'].mode())
print('Sex frequent:', train['Sex'].mode())

Pclass frequent: 0    3
dtype: int64
Sex frequent: 0    1.0
dtype: float64


In [11]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    float64
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    float64
dtypes: float64(4), int64(5), object(3)
memory usage: 83.7+ KB


In [12]:
from sklearn.model_selection import StratifiedShuffleSplit
split = StratifiedShuffleSplit(test_size = 0.1, random_state = 3)
for train_index, test_index in split.split(train, train['Pclass']):
    strat_train_set = train.loc[train_index]
    strat_test_set = train.loc[test_index]

In [13]:
import matplotlib.pyplot as plt
def hist_plot(data, feature, bins):
    data[feature].hist(bins = bins, figsize =(20,15))
    plt.show()

In [14]:
trainx = strat_train_set.loc[:, strat_train_set.columns != 'Survived']
testx = strat_test_set.loc[:, strat_test_set.columns != 'Survived']
print(trainx.shape)
trainx.head()

(801, 11)


,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
188,189,3,"Bourke, Mr. John",1.0,40.0,1,1,364849,15.50,NaN,1.0
170,171,1,"Van der hoef, Mr. Wyckoff",1.0,61.0,0,0,111240,33.50,B19,2.0
148,149,2,"Navratil, Mr. Michel (""Louis M Hoffman"")",1.0,36.5,0,2,230080,26.00,F2,2.0
845,846,3,"Abbing, Mr. Anthony",1.0,42.0,0,0,C.A. 5547,7.55,NaN,2.0
290,291,1,"Barber, Miss. Ellen ""Nellie""",0.0,26.0,0,0,19877,78.85,NaN,2.0


In [15]:
trainy = strat_train_set.loc[:,'Survived'].to_frame()
testy = strat_test_set.loc[:, 'Survived'].to_frame()
print(trainy.shape)
trainy.head()

(801, 1)


,Survived
188,0
170,0
148,0
845,0
290,1


In [16]:
trainx['Sex'].value_counts()

1.0    514
0.0    287
Name: Sex, dtype: int64

Finding correlation b/w features and target variable('Survived')

In [17]:
train.corr()['Survived']

PassengerId   -0.005007
Survived       1.000000
Pclass        -0.338481
Sex           -0.543351
Age           -0.077221
SibSp         -0.035322
Parch          0.081629
Fare           0.257307
Embarked      -0.169718
Name: Survived, dtype: float64

Valuable correlation of features with survival: Pclass, Embarked and Sex - negatively correlated; Fare - positively correlated; but not to significant extent 

In [18]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(random_state = 3).fit(trainx[['Pclass', 'Sex', 'Fare']], trainy.values.ravel())

In [19]:
prediction = clf.predict(testx[['Pclass', 'Sex', 'Fare']])

In [20]:
prediction.shape

(90,)

In [21]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(testy, prediction, normalize = True)
accuracy

0.7666666666666667

Accuracy of a simple logistic regression employing three features Pclass, Sex and Fare

In [22]:
test = pd.read_csv('/kaggle/input/titanic/test.csv')
print(test.shape)
test.head()

(418, 11)


,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [23]:
test_stats = test.describe(include = 'all')

In [24]:
def prediction(test):
    test['Sex'] = enc.fit_transform(test.loc[:, test.columns == 'Sex'])
    test['Pclass'].fillna(test['Pclass'].mode(), inplace = True)
    test['Sex'].fillna(test['Sex'].mode(), inplace = True)
    test['Fare'].fillna(test['Fare'].mean(), inplace = True)
    predict = clf.predict(test[['Pclass', 'Sex', 'Fare']])
    return predict

In [25]:
test_predictions = prediction(test)
test_predictions

array([0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0,
       1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1,
       1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1,
       1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1,
       0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0,
       1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
       0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0,

In [26]:
test_predictions2 = prediction(test)
np.sum(test_predictions != test_predictions2)

0

In [27]:
submission = pd.DataFrame()

In [28]:

submission['PassengerId'] = test['PassengerId']
submission['Survived'] = test_predictions
submission.to_csv("submission.csv", index = False)